# Thalamic nuclei mean kurtosis estimation

> Import ROIs

In [13]:
import pandas as pd
import os
import re
import numpy as np
import nibabel as nb
import argparse
import textwrap

> Set input subjects / file locations

In [4]:
dataLoc = os.getcwd()
subjects = [x for x in os.listdir(dataLoc) if x.startswith('N') or x.startswith('F')]

In [34]:
def get_segment_map(subject,side):
    '''
    Returns nibabel matrix of 
    the connectivity-based segmentation map of thalamus
    '''
    segment_file = os.path.join(dataLoc,
                              subject,
                              'segmentation',
                              side,
                              'dki_biggest.nii.gz')
    f = nb.load(segment_file)
    segment_map = f.get_data()
    return segment_map

In [35]:
def get_mk_map(subject):
    '''
    Returns nibabel matrix of
    the whole brain mean kurtosis image
    '''
    mk_file = os.path.join(dataLoc,
                         subject,
                         'DKI',
                         'kmean.nii')
    f = nb.load(mk_file)
    mk_map = f.get_data()
    return mk_map

In [36]:
def get_matrix_for_nuclei(segment_map, mk_map, cortex):
    '''
    Returns mean of the MK 
    of the thalamic segments 
    in connection to sepcific cortex
    '''
    nuclei_dict = {"LPFC":1,
                  "LTC":2,
                  "MPFC":3,
                  "MTC":4,
                  "OCC":5,
                  "OFC":6,
                  "PC":7,
                  "SMC":8}
    nucleiNum = nuclei_dict[cortex]
    cortexMap = segment_map != nucleiNum
    segment_mk = cortexMap * mk_map
    mk_mean = segment_mk[segment_mk!=0].mean()
    return mk_mean

In [46]:
def thalamic_mk(subject, side):
    '''
    Returns a pandas dataframe,
    containing MK of thalamic segments 
    for a given subject
    '''
    group = subject[:3]
    segment_map = get_segment_map(subject, side)
    mk_map = get_mk_map(subject)
    
    df = pd.DataFrame()
    for cortex in ["LPFC", "LTC", "MPFC", "MTC", "OCC", "OFC", "PC", "SMC"]:
        mk_mean = get_matrix_for_nuclei(segment_map, mk_map, cortex)
        cortex_df = pd.DataFrame(data = [[mk_mean, cortex, side, group]], 
                                 index=[subject], 
                                 columns=['MK','cortex','side','group'])
        df = pd.concat([df, cortex_df])
    
    return df

In [47]:
thalamic_mk('NOR26_YJH','left')

,MK,cortex,side,group
NOR26_YJH,0.760900557041,LPFC,left,NOR
NOR26_YJH,0.761007130146,LTC,left,NOR
NOR26_YJH,0.760981142521,MPFC,left,NOR
NOR26_YJH,0.760972559452,MTC,left,NOR
NOR26_YJH,0.760971009731,OCC,left,NOR
NOR26_YJH,0.760947942734,OFC,left,NOR
NOR26_YJH,0.760874271393,PC,left,NOR
NOR26_YJH,0.760946035385,SMC,left,NOR


In [48]:
thalamic_mk('NOR81_KSA','left')

,MK,cortex,side,group
NOR81_KSA,0.783796429634,LPFC,left,NOR
NOR81_KSA,0.783984780312,LTC,left,NOR
NOR81_KSA,0.783967316151,MPFC,left,NOR
NOR81_KSA,0.784175932407,MTC,left,NOR
NOR81_KSA,0.783983826637,OCC,left,NOR
NOR81_KSA,0.783742904663,OFC,left,NOR
NOR81_KSA,0.783677279949,PC,left,NOR
NOR81_KSA,0.78398835659,SMC,left,NOR


In [42]:
def main():
    '''
    Estimates the mean kurtosis in the thalamic segments
    for all subjects.
    The thalamic segments is obtained from the connectivity-based segmentation
    from FSL.
    
    It returns mk_data.csv containing all subjects' data.
    
    '''
    allData = pd.DataFrame()
    for subject in subjects:
        for side in ['left','right']:
            subjectDf = thalamic_mk(subject, side)
            allData = pd.concat([allData, subjectDf])
    
    allData.to_csv('mk_data.csv')

```
if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter,
        description=textwrap.dedent('''\
            {codeName} : Estimates mean kurtosis in the thalamic nuclei
            ===========================================================
            '''.format(codeName=os.path.basename(__file__))))

    parser.add_argument(
        '-s', '--subjects',
        help='subject list',
        nargs='+')

    parser.add_argument(
        '-side', '--side',
        help='side')
    
    args = parser.parse_args()    
    thalamic_mk(args)
```